# Naming convention and contents of the created files

1. `faket/class_mask.mrc` = `class_mask.mrc` sliced to valid region.
1. `faket/occupancy_mask.mrc` = `occupancy_mask.mrc` slice to valid region.
1. `reconstruction.mrc` = `faket/reconstruction_shrec.mrc` slice to valid region for tomogram 9.
1. `faket/projections_noiseless.mrc` = `grandmodel_unbinned.mrc` measured with Radon transform.
1. `faket/projections_content.mrc` = `faket/projections_noiseless.mrc` + noise (std=0.1) shifted & scaled according its style*.
1. `faket/projections_noisy.mrc` = `faket/projections_noiseless.mrc` + noise (std=0.4) shifted & scaled according its style*.
1. `faket/projections_styled.mrc` = result of NST initialized with `faket/projections_noisy.mrc`, content `faket/projections_content.mrc`, and using its style*.
1. `faket/reconstruction_content.mrc` = reconstruction of `faket/projections_content.mrc`.
1. `faket/reconstruction_noisy.mrc` = reconstruction of `faket/projections_noisy.mrc`.
1. `faket/reconstruction_styled.mrc` = reconstruction of `faket/projections_styled.mrc`.
1. `faket/reconstruction_baseline.mrc` = reconstruction of `projections.mrc`.

\* Each time we mention style in the text above, it refers to a `projections.mrc` file from a model_N+1. In case N=8, the style is taken from N=0.

In [1]:
%load_ext autoreload
%autoreload 2

from os.path import join as pj
import multiprocessing
import gpuMultiprocessing
from faket.data import load_mrc, save_mrc, save_conf
from faket.data import slice_to_valid, vol_to_valid
from faket.data import downsample_sinogram_space
from faket.data import get_clim, get_theta
from faket.data import match_mean_std, normalize
from faket.transform import radon_3d, reconstruct_mrc
import matplotlib.pyplot as plt
# from tqdm.notebook import tqdm
import numpy as np

In [ ]:
data_folder = 'data/shrec2021_extended_dataset/'

In [ ]:
# SHREC21 provides the data in square shape even
# thought the data is stored only in the center
# The following values specify where to slice
z_valid = (0.32226, 0.67382)  # Valid range normalized

In [ ]:
# slice class_mask.mrc to faket/class_mask.mrc
for N in range(10):
    vol_to_valid(data_folder, N, 'class_mask', z_valid, 
                 out_fname='faket/class_mask.mrc')

In [ ]:
# slice occupancy_mask.mrc to faket/occupancy_mask.mrc
for N in range(10):
    vol_to_valid(data_folder, N, 'occupancy_mask', z_valid, 
                 out_fname='faket/occupancy_mask.mrc')

In [ ]:
# slice reconstruction.mrc to faket/reconstruction_shrec.mrc
vol_to_valid(data_folder, 9, 'reconstruction', z_valid, 
                 out_fname='faket/reconstruction_shrec.mrc')

## Creating projections

In [ ]:
# create faket/projections_noiseless.mrc by measuring the grandmodel_unbinned.mrc with Radon transform

for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = {
        'input_mrc': pj(data_folder, f'model_{N}', 'grandmodel_unbinned.mrc'),
        'output_mrc': pj(data_folder, f'model_{N}', 'faket/projections_noiseless.mrc'),
        'radon_kwargs': {
            'theta': get_theta(data_folder, N),
            'dose': 0,
            'out_shape': 1024,
            'slice_axis': 1,
            # circle=False because we measure with the data outside the circle 
            # but later we cut the measurements to desired shape 
            # SHREC did it this way - confirmed from a personal communication
            'circle': False
        }
    }
    volume = load_mrc(conf['input_mrc'])
    sinogram = radon_3d(volume, **conf['radon_kwargs'])
    save_conf(conf['output_mrc'], conf)
    save_mrc(sinogram.astype(np.float32), conf['output_mrc'], overwrite=True)
print('Done')

In [ ]:
# create faket/projections_content.mrc and faket/projections_noisy.mrc
for N in range(0, 1): # We do not need this modality for the test model_9
    print(f'Processing N: {N}')
    style_N = (N + 1) % 9 # For the last train model we take style stats from the first train model
    
    conf = {  # For noisy
        'input_mrc': pj(data_folder, f'model_{N}', 'faket/projections_noiseless.mrc'),
        'style_mrc': pj(data_folder, f'model_{style_N}', 'projections.mrc'),
        'output_mrc': pj(data_folder, f'model_{N}', 'faket/projections_noisy.mrc'),
        'mean': 0.0,
        'std': 0.4,
        'clip_outliers': (0.0001, 0.9999),
        'seed': N,
    }
    
    conf2 = {  # Changes for content
        'output_mrc': pj(data_folder, f'model_{N}', 'faket/projections_content.mrc'),
        'std': 0.1,
    }
    
    volume = load_mrc(conf['input_mrc'])
    style = load_mrc(conf['style_mrc'])
    
    rng = np.random.default_rng(seed=conf['seed'])
    noise = rng.normal(loc=conf['mean'], scale=conf['std'], 
                       size=volume.size).reshape(volume.shape)
    
    volume  = match_mean_std(volume, style)  # Scaling per tilt (bigger the abs(angle), longer the trajectory)
    volume = normalize(volume)  # Scale between [0, 1]
    
    volume_noisy = volume + noise
    volume_noisy = np.clip(volume_noisy, *get_clim(volume_noisy, *conf['clip_outliers']))  # Remove outliers
    volume_noisy = match_mean_std(volume_noisy, style)  # Scale back to match style
    
    save_conf(conf['output_mrc'], conf)
    save_mrc(volume_noisy.astype(np.float32), conf['output_mrc'], overwrite=True)
    
    ratio = conf['std'] / conf2['std']
    conf.update(conf2)
    volume_content = volume + noise / ratio  # Same noise just a fraction of the std
    volume_content = np.clip(volume_content, *get_clim(volume_content, *conf['clip_outliers']))  # Remove outliers
    volume_content = match_mean_std(volume_content, style)  # Scale back to match style
    
    save_conf(conf['output_mrc'], conf)
    save_mrc(volume_content.astype(np.float32), conf['output_mrc'], overwrite=True)
print('Done')

### Neural Style Transfer

In [ ]:
nstc = {  # NEURAL STYLE TRANSFER BASE CONFIG
    # 'content': 'example.mrc',
    # 'style': 'example.mrc',
    # '--init': 'example.mrc',
    # '--output': 'example.mrc', 
    # '--random-seed': None,
    '--style-weights': 1.0,
    '--content-weight': 1.0, 
    '--tv-weight': 0,
    '--min-scale': 1024,
    '--end-scale': 1024,
    '--iterations': 1,
    '--initial-iterations': 1,
    '--save-every': 2,
    '--step-size': 0.15,
    '--avg-decay': 0.99,
    '--style-scale-fac': 1.0,
    '--pooling': 'max',
    '--devices': 'cuda:0', #
    '--seq_start' : 0,
    '--seq_end' : 61,
}

def get_command(expname, nst_command, config):
    command = (
    f"EXPNAME={expname} {nst_command} "
    f"{config['content']} {config['style']} "
    f"{' '.join([f'{k} {v}' for k, v in config.items() if k.startswith('--')])}")
    return command

In [ ]:
# create faket/projections_styled.mrc
gpu_id_list = [0]
NST_command = 'python3 -m faket.style_transfer.cli'

command_queue = []
for N in range(0, 10): # We do not need this modality for the test model_9
    style_N = (N + 1) % 9 # For the last train model we take style stats from the first train model
    
    EXPNAME = f'TOMOGRAM_{N}'  # Just for visualizing the progress
    tomo_folder = pj(data_folder, f'model_{N}', 'faket')

    conf = nstc.copy()
    conf.update({
        'content': pj(tomo_folder, 'projections_content.mrc'),
        'style': pj(data_folder, f'model_{style_N}', 'projections.mrc'), 
        '--init': pj(tomo_folder, 'projections_noisy.mrc'),
        '--output': pj(tomo_folder, 'projections_styled.mrc'), 
        '--random-seed': N,
    })
    
    command = get_command(EXPNAME, NST_command, conf)
    command_queue.append(command)
    
# Run all the commands (returns list of failed commands if any)
gpuMultiprocessing.queue_runner(command_queue, gpu_id_list,
                                env_gpu_name='CUDA_VISIBLE_DEVICES',
                                processes_per_gpu=6, allowed_restarts=1)

## Computing reconstructions

In [ ]:
recc = {  # RECONSTRUCTION BASE CONFIG
    'downsample_angle' : 1,  # Sinogram downsampling in theta dimension (1 = no downsampling)
    'downsample_pre' : 2,  # Sinogram downsampling (1 = no downsampling)
    'order' : 3,  # Downsampling in space with spline interpolation of order (0 - 5)
    'filtering' : 'ramp2d',  # Filter userd during reconstruction in FBP algorithm
    'filterkwargs' : {'crowtherFreq': 25, 'radiusCutoff': 230, 'angularCutoff': (0, 83)},
    'downsample_post' : 1,  # Reconstruction downsampling
    'ncpus': 61, # multiprocessing.cpu_count(),  # Number of CPUs to use while reconstructing
    'z_valid': z_valid # 2-tuple range of valid pixels in Z dimension normalized from 0 to 1. (0., 1.) or None for all.
}

In [ ]:
# reconstruct faket/projections_content.mrc to produce faket/reconstruction_content.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' :  pj(data_folder, f'model_{N}', 'faket/projections_content.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' :  pj(data_folder, f'model_{N}', 'faket/reconstruction_content.mrc')
    })
    reconstruct_mrc(**conf)

In [ ]:
# reconstruct faket/projections_noisy.mrc to produce faket/reconstruction_noisy.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' : pj(data_folder, f'model_{N}', 'faket/projections_noisy.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' : pj(data_folder, f'model_{N}', 'faket/reconstruction_noisy.mrc')
    })
    reconstruct_mrc(**conf)

In [ ]:
# reconstruct faket/projections_styled.mrc to produce faket/reconstruction_styled.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' : pj(data_folder, f'model_{N}', 'faket/projections_styled.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' : pj(data_folder, f'model_{N}', 'faket/reconstruction_styled.mrc')
    })
    reconstruct_mrc(**conf)

In [ ]:
# reconstruct projections.mrc to produce faket/reconstruction_baseline.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' :  pj(data_folder, f'model_{N}', 'projections.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' : pj(data_folder, f'model_{N}', 'faket/reconstruction_baseline.mrc')
    })
    reconstruct_mrc(**conf)

In [ ]:
# reconstruct faket/projections_noiseless.mrc to produce faket/reconstruction_noiseless.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' :  pj(data_folder, f'model_{N}', 'faket/projections_noiseless.mrc'), 
        'theta': pj(data_folder, f'model_{N}', 'alignment_simulated.txt'), 
        'output_mrc' : pj(data_folder, f'model_{N}', 'faket/reconstruction_noiseless.mrc')
    })
    reconstruct_mrc(**conf)

# Deep Finder experiments

1. `DF('faket/reconstruction_baseline.mrc')` - train for 30 epochs on 9 tomograms, eval on test tomogram every 5 epochs
2. `DF('faket/reconstruction_content.mrc')` - train for 30 epochs on 9 tomograms, eval on test tomogram every 5 epochs
3. `DF('faket/reconstruction_noisy.mrc')` - train for 30 epochs on 9 tomograms, eval on test tomogram every 5 epochs
4. `DF('faket/reconstruction_styled.mrc')` - train for 30 epochs on 9 tomograms, eval on test tomogram every 5 epochs

In [ ]:
# OUTDATED
# run segmentation step on GPU with ID GPU_no
!python3 faket/deepfinder/step1_launch_segment_loop.py --path_config results/

In [ ]:
# OUTDATED
# run clustering 
!python3 faket/deepfinder/step2_clustering_loop.py --path_config results/

epoch 1/2 - it 322/688 - loss: 13.331 - acc: 0.899
epoch 1/2 - it 323/688 - loss: 13.116 - acc: 0.902
epoch 1/2 - it 324/688 - loss: 13.119 - acc: 0.906
epoch 1/2 - it 325/688 - loss: 13.547 - acc: 0.909
epoch 1/2 - it 326/688 - loss: 13.212 - acc: 0.893
epoch 1/2 - it 327/688 - loss: 13.267 - acc: 0.896
epoch 1/2 - it 328/688 - loss: 13.071 - acc: 0.881
epoch 1/2 - it 329/688 - loss: 13.305 - acc: 0.899
epoch 1/2 - it 330/688 - loss: 13.062 - acc: 0.905
epoch 1/2 - it 331/688 - loss: 13.372 - acc: 0.896
epoch 1/2 - it 332/688 - loss: 13.118 - acc: 0.904
epoch 1/2 - it 333/688 - loss: 12.979 - acc: 0.906
epoch 1/2 - it 334/688 - loss: 13.045 - acc: 0.888
epoch 1/2 - it 335/688 - loss: 13.028 - acc: 0.904
epoch 1/2 - it 336/688 - loss: 13.088 - acc: 0.890
epoch 1/2 - it 337/688 - loss: 12.895 - acc: 0.912
epoch 1/2 - it 338/688 - loss: 13.054 - acc: 0.902
epoch 1/2 - it 339/688 - loss: 13.075 - acc: 0.904
epoch 1/2 - it 340/688 - loss: 13.112 - acc: 0.905
epoch 1/2 - it 341/688 - loss: 

Traceback (most recent call last):
  File "/home/herrmannl22/test_faket/faket/faket/deepfinder/launch_training.py", line 111, in <module>
    trainer.launch(path_data, path_target, objl_train, objl_valid)
  File "/home/herrmannl22/test_faket/faket/faket/deepfinder/training.py", line 292, in launch
    loss_train = self.net.train_on_batch(batch_data, batch_target,
  File "/home/herrmannl22/.conda/envs/faket2/lib/python3.9/site-packages/keras/engine/training.py", line 1852, in train_on_batch
    iterator = data_adapter.single_batch_iterator(self.distribute_strategy, x,
  File "/home/herrmannl22/.conda/envs/faket2/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 1625, in single_batch_iterator
    x, y, sample_weight = _process_tensorlike((x, y, sample_weight))
  File "/home/herrmannl22/.conda/envs/faket2/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 1031, in _process_tensorlike
    inputs = tf.nest.map_structure(_convert_numpy_and_scipy, inputs)
  File "/h

In [ ]:
# OUTDATED
# run evaluation
!python3 faket/deepfinder/step3_launch_evaluation.py --path_config results/

In [2]:
# OUTDATED
train_script="""
for idx in 1 2 3 4; 
do 
    PYTHONHASHSEED=0 python3 faket/deepfinder/launch_training.py --path_config results/experiment_${idx}a/
done
"""
with open('train_script.sh', 'w') as file:
    file.write(train_script)

!bash train_script.sh

2022-02-18 14:45:15.317554: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-18 14:45:16.478774: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-02-18 14:45:16.478861: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1371 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
Loading dataset ...
Launch training ...
2022-02-18 14:45:19.493696: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization 

In [2]:
import os
from itertools import product

In [3]:
def get_full_DF_training_command(tomo_path, DF_training_command, config):
    command = (
        f"{DF_training_command} "
        f"--training_tomogram_ids {' '.join(list(zip(*config['training_tomograms']))[0])} "
        f"--training_tomograms {' '.join(list(zip(*config['training_tomograms']))[1])} "
        f"{' '.join([f'{k} {v}' for k, v in config.items() if k.startswith('--')])} "       
    )
    return command

In [9]:
gpu_id_list = [0]
tomo_path = 'data/shrec2021_extended_dataset/'
DF_training_command ='PYTHONHASHSEED=0 python faket/deepfinder/launch_training.py'

# create config files from dict using json for different seeds
experiment_name = "exp1"
num_seeds = 1

command_queue = []

for N in range(1,num_seeds+1):
    training_conf ={
        "--training_tomo_path" : tomo_path,
        "training_tomograms" : [[str(i),"baseline"] for i in range(0,9)],
        "--num_epochs" : 2,
        "--out_path" : "results/" + experiment_name + "/seed" + str(N) + "/",
        "--save_every" : 1, 
        "--seed1" : "1" + str(N) + "2",
        "--seed2" : "1" + str(N) + "23",
    }
    
    command = get_full_DF_training_command(tomo_path, DF_training_command, training_conf)
    command_queue.append(command)

    
# Run all the commands (returns list of failed commands if any)
gpuMultiprocessing.queue_runner(command_queue, gpu_id_list,
                                env_gpu_name='CUDA_VISIBLE_DEVICES',
                                processes_per_gpu=2, allowed_restarts=0)

2022-02-21 13:35:08.848449: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 13:35:09.521435: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-02-21 13:35:09.521510: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38228 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2022-02-21 13:35:12.956849: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-

Loading dataset ...
Launch training ...
epoch 1/2 - it 1/688 - loss: 15.716 - acc: 0.011
epoch 1/2 - it 2/688 - loss: 15.724 - acc: 0.020
epoch 1/2 - it 3/688 - loss: 15.714 - acc: 0.054
epoch 1/2 - it 4/688 - loss: 15.706 - acc: 0.130
epoch 1/2 - it 5/688 - loss: 15.713 - acc: 0.283
epoch 1/2 - it 6/688 - loss: 15.701 - acc: 0.460
epoch 1/2 - it 7/688 - loss: 15.699 - acc: 0.629
epoch 1/2 - it 8/688 - loss: 15.678 - acc: 0.752
epoch 1/2 - it 9/688 - loss: 15.691 - acc: 0.853
epoch 1/2 - it 10/688 - loss: 15.663 - acc: 0.863
epoch 1/2 - it 11/688 - loss: 15.666 - acc: 0.901
epoch 1/2 - it 12/688 - loss: 15.612 - acc: 0.887
epoch 1/2 - it 13/688 - loss: 15.544 - acc: 0.878
epoch 1/2 - it 14/688 - loss: 15.426 - acc: 0.875
epoch 1/2 - it 15/688 - loss: 15.308 - acc: 0.886
epoch 1/2 - it 16/688 - loss: 15.146 - acc: 0.904
epoch 1/2 - it 17/688 - loss: 15.071 - acc: 0.888
epoch 1/2 - it 18/688 - loss: 15.049 - acc: 0.892
epoch 1/2 - it 19/688 - loss: 15.043 - acc: 0.885
epoch 1/2 - it 20/6

KeyboardInterrupt: 

In [22]:
def get_full_DF_analysis_command(DF_weight_path, DF_analysis_command, config):
    command=(
        f"{DF_analysis_command} "
        f"{' '.join([f'{k} {v}' for k, v in config.items() if k.startswith('--')])} "
    )
    return command

In [16]:
gpu_id_list = [0]
DF_segmentation_command ='python faket/deepfinder/launch_segmentation.py'
DF_weight_path = ''

experiment_name = "exp1"
seed_ids = [1, 2]
num_epochs = [1,2]
test_tomograms = ["baseline"]
test_tomograms_idx = 9

command_queue = []

for N, num_epoch, test_tomogram in product(seed_ids, num_epochs, test_tomograms):
    analysis_conf ={
        "--test_tomo_path" : tomo_path,
        "--test_tomo_idx" : 9, 
        "--test_tomogram" : test_tomogram,
        "--num_epochs" : num_epoch,
        "--DF_weights_path" : "results/" + experiment_name + "/seed" + str(N) + "/",
        "--out_path" : "results/" + experiment_name + "/seed" + str(N) + "/", 
    }
    
    command = get_full_DF_analysis_command(DF_weight_path, DF_segmentation_command, analysis_conf)
    command_queue.append(command)
print(command_queue)


    
# Run all the commands (returns list of failed commands if any)
gpuMultiprocessing.queue_runner(command_queue, gpu_id_list,
                                env_gpu_name='CUDA_VISIBLE_DEVICES',
                                processes_per_gpu=2, allowed_restarts=0)

['python faket/deepfinder/launch_segmentation.py --test_tomo_path data/shrec2021_extended_dataset/ --test_tomo_idx 9 --test_tomogram baseline --num_epochs 1 --DF_weights_path results/exp1/seed1/ --out_path results/exp1/seed1/ ', 'python faket/deepfinder/launch_segmentation.py --test_tomo_path data/shrec2021_extended_dataset/ --test_tomo_idx 9 --test_tomogram baseline --num_epochs 2 --DF_weights_path results/exp1/seed1/ --out_path results/exp1/seed1/ ', 'python faket/deepfinder/launch_segmentation.py --test_tomo_path data/shrec2021_extended_dataset/ --test_tomo_idx 9 --test_tomogram baseline --num_epochs 1 --DF_weights_path results/exp1/seed2/ --out_path results/exp1/seed2/ ', 'python faket/deepfinder/launch_segmentation.py --test_tomo_path data/shrec2021_extended_dataset/ --test_tomo_idx 9 --test_tomogram baseline --num_epochs 2 --DF_weights_path results/exp1/seed2/ --out_path results/exp1/seed2/ ']


2022-02-21 10:13:22.410961: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 10:13:22.412978: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 10:13:23.198752: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-02-21 10:13:23.198816: I tensorflow/core/common_runtime/g

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/herrmannl22/test_faket/faket/faket/deepfinder/launch_segmentation.py", line 59, in <module>
    scoremaps = seg.launch(tomo)
  File "/home/herrmannl22/test_faket/faket/faket/deepfinder/segmentation.py", line 97, in launch
    :] + np.float16(pred[0,
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/herrmannl22/test_faket/faket/faket/deepfinder/launch_segmentation.py", line 59, in <module>
    scoremaps = seg.launch(tomo)
  File "/home/herrmannl22/test_faket/faket/faket/deepfinder/segmentation.py", line 97, in launch
    :] + np.float16(pred[0,
KeyboardInterrupt


Data array is divided in 50 patches ...
Segmenting patch 1 / 50 ...
Segmenting patch 2 / 50 ...
Segmenting patch 3 / 50 ...
Segmenting patch 4 / 50 ...
Segmenting patch 5 / 50 ...
Segmenting patch 6 / 50 ...
Segmenting patch 7 / 50 ...
Segmenting patch 8 / 50 ...
Segmenting patch 9 / 50 ...
Segmenting patch 10 / 50 ...
Data array is divided in 50 patches ...
Segmenting patch 1 / 50 ...
Segmenting patch 2 / 50 ...
Segmenting patch 3 / 50 ...
Segmenting patch 4 / 50 ...
Segmenting patch 5 / 50 ...
Segmenting patch 6 / 50 ...
Segmenting patch 7 / 50 ...
Segmenting patch 8 / 50 ...
Segmenting patch 9 / 50 ...
Segmenting patch 10 / 50 ...


In [6]:
from multiprocessing import Process


def multiprocessing_analysis(fct, queue, num_processes):
    """
    Starts the processes in parallel of fct called with commands from queue
    with at most num_processes at the same time
    """
    while len(queue) > 0:
        processes = []
        jobs_left = len(queue)
        for _ in range(min(num_processes, jobs_left)):
            argument_dict = queue.pop(0)
            p = Process(target=fct, args=(argument_dict["test_tomogram"],
                                          argument_dict["test_tomo_idx"], 
                                          argument_dict["num_epochs"],
                                          argument_dict["label_map_path"],
                                          argument_dict["out_path"]))
            p.start()
            processes.append(p)
        
        for p in processes:
            p.join()
            

In [7]:
# after segmentation finishes, the clustering and evaluation step can be started
from faket.deepfinder.launch_clustering import launch_clustering
from faket.deepfinder.launch_evaluation import launch_evaluation
num_processes = 2

queue = []

for N, num_epoch, test_tomogram in product(seed_ids, num_epochs, test_tomograms):
    analysis_conf ={
        "test_tomogram" : test_tomogram,
        "test_tomo_idx" : 9,
        "num_epochs" : num_epoch,
        "label_map_path" : "results/" + experiment_name + "/seed" + str(N) + "/",
        "out_path" : "results/" + experiment_name + "/seed" + str(N) + "/", 
    }
    
    queue.append(analysis_conf)


multiprocessing_analysis(lambda v,w,x,y,z: launch_clustering(v,w,x,y,z), queue, num_processes)
multiprocessing_analysis(lambda v,w,x,y,z: launch_evaluation(v,w,x,y,z), queue, num_processes)

results/exp1/seed1/tomo9_baseline_2021_1epoch_bin1_objlist_thr.xml
results/exp1/seed1/tomo9_baseline_2021_2epoch_bin1_objlist_thr.xml
Processing submission #0: results/exp1/seed1/particle_locations_tomo_baseline_.txt


	### Localization
	Submission has 2133 predicted particles
	Tomogram has 1571 particles
	TP: 1127 unique particles found
	FP: 670 predicted particles are false positive
	FN: 438 unique particles not found
	There was 12 particles that had more than one prediction
	There was 0 predicted particles that were outside of tomo bounds
	Average euclidean distance from predicted center to ground truth center: 165.43723544865617
	Recall: 0.71738
	Precision: 0.52836
	Miss rate: 0.28262
	F1 score: 0.60853

	### Classification
	Predict          1BXN             1QVR             1S3X             1U6G             2CG9             3CF3             3D2F             3GL1             3H84             3QM1             4CR2             5MRC             background       fiducial         
	Actu

Process Process-43:
Process Process-44:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/herrmannl22/.conda/envs/faket2/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/herrmannl22/.conda/envs/faket2/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/herrmannl22/.conda/envs/faket2/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/herrmannl22/.conda/envs/faket2/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_3413812/177846917.py", line 21, in <lambda>
    multiprocessing_analysis(lambda v,w,x,y,z: launch_evaluation(v,w,x,y,z), queue, num_processes)
  File "/tmp/ipykernel_3413812/177846917.py", line 21, in <lambda>
    multiprocessing_analysis(lambda v,w,x,y,z: launch_evaluation(v,w,x,y,z), queue, num_processes)
  

KeyboardInterrupt: 